<a href="https://colab.research.google.com/github/zamalex/BakingFinal/blob/master/k%20fold%20uci%20har.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import pickle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import sklearn.metrics
from tensorflow.keras.utils import plot_model
import datetime


In [2]:
%load_ext tensorboard


# Some methods and data which will be used below

In [3]:
# (i).   Mapping labels to their resp. classes
# (ii).  Mapping classes to their resp. labels

label_to_class = {
    1  : 'WALKING',
    2  : 'WALKING_UPSTAIRS',
    3  : 'WALKING_DOWNSTAIRS',
    4  : 'SITTING',
    5  : 'STANDING',
    6  : 'LAYING',
    7  : 'STAND_TO_SIT',
    8  : 'SIT_TO_STAND',
    9  : 'SIT_TO_LIE',
    10 : 'LIE_TO_SIT',
    11 : 'STAND_TO_LIE',
    12 : 'LIE_TO_STAND',
    np.nan : np.nan
}
class_to_label = {
    'WALKING' : 1,
    'WALKING_UPSTAIRS' : 2,
    'WALKING_DOWNSTAIRS' : 3,
    'SITTING' : 4,
    'STANDING' : 5,
    'LAYING' : 6,
    'STAND_TO_SIT' : 7,
    'SIT_TO_STAND' : 8,
    'SIT_TO_LIE' : 9,
    'LIE_TO_SIT' : 10,
    'STAND_TO_LIE' : 11,
    'LIE_TO_STAND' : 12,
    np.nan : np.nan
}

In [4]:
# (i).   Removing data-points where y and x values is null

# Other methods can be
# ffill (forward fill) => fills using forward points
# bfill (backward fill) => using backward points
# interpolate

def remove_null(xdata,ydata):
    xdata = xdata[np.where(np.isfinite(ydata))]
    ydata = ydata[np.where(np.isfinite(ydata))]
    ydata = ydata[np.where(np.isfinite(xdata).all(axis = 1).all(axis = 1).all(axis = 1))]
    xdata = xdata[np.where(np.isfinite(xdata).all(axis = 1).all(axis = 1).all(axis = 1))]

    return xdata,ydata

In [5]:
# normalize xdata using sklearn.preprocessing.StandardScaler and returns
# scaler object to use it furthur for testing data

# Each axis of each sensor has different min, max, I scaled according to them seperately
# Initial shape == (None,128,2,3)
# changed to (None , 6) :-
# reshape to (None,128,6) -> swapaxis(0,2) -> reshape(6,-1) -> transpose
# Fit scaler OR transform according to scaler

# Reverse above process to get back oiginal data
# transpose -> reshape(6,128,None) -> swapaxes(0,2) -> reshape(None,128,2,3)

def get_scaler(xdata):

    row = xdata.shape[0]
    timestamp = xdata.shape[1]
    sensor = xdata.shape[2]
    axis = xdata.shape[3]

    scaler = MinMaxScaler(feature_range = (-1,1))
    xdata = xdata.reshape(row,timestamp,sensor*axis)
    xdata = np.swapaxes(xdata,0,2).reshape(sensor*axis,-1).T
    scaler.fit(xdata)
    return scaler

def scale_data(xdata,scaler):

    row = xdata.shape[0]
    timestamp = xdata.shape[1]
    sensor = xdata.shape[2]
    axis = xdata.shape[3]

    xdata = xdata.reshape(row,timestamp,sensor*axis)
    xdata = np.swapaxes(xdata,0,2).reshape(sensor*axis,-1).T
    xdata = scaler.transform(xdata)
    xdata = xdata.T.reshape(sensor*axis,timestamp,row)
    xdata = np.swapaxes(xdata,0,2).reshape(row,timestamp,sensor,axis)

    return xdata

In [6]:
# takes in location, exp no., user no., start and end(end point is excluded from reading i.e lastpoint+1) point
# ,overlap array, and returns xdata and ydata

def create_windows(location, exp, user, start, end, activity, length, overlap):

    acc_file  = location + '/acc_exp'+ str(exp).zfill(2) + '_user' + str(user).zfill(2) + '.txt'
    gyro_file = location + '/gyro_exp'+ str(exp).zfill(2) + '_user' + str(user).zfill(2) + '.txt'

    acc_data  = np.loadtxt(acc_file)
    gyro_data = np.loadtxt(gyro_file)

    xtrain = []
    ytrain = []

    while (start + length <= end) :

        stop = start + length
        window = []

        while start != stop :
            window.append( [acc_data[start] , gyro_data[start]] )
            start += 1

        xtrain.append(window)
        ytrain.append(activity)

        start = stop - overlap[activity-1]

    return xtrain,ytrain

In [7]:
# location == location of file
# lenght == lenght of window
# overlap == array of overlaps of size == number of unique activities
# overlap depends on activity so as to extract more data from a particular class if needed


# (i).   Loading labels.txt as labels
# (ii).  Iterating in labels and calling create_windows on acceleration file, extending returned data in xtrain, ytrain
# (iii). Iterating in labels and calling create_windows on gyroscope file, extending returned data in xtrain, ytrain

def prepare_data(location,length = 128,overlap = [64]*12):

    xdata = []
    ydata = []

    labels = np.loadtxt(location+'/labels.txt',dtype = 'uint32')

    for exp,user,activity,start,end in labels :

        xtemp , ytemp = create_windows(location, exp, user, start, end+1, activity, length, overlap)
        xdata.extend(xtemp)
        ydata.extend(ytemp)

    return np.array(xdata),np.array(ydata)

In [8]:
# (i). Finds max element index sets its 1 and sets remaining 0
#      for each row

def to_categorical(ydata):

    for i in range(len(ydata)):
        j = ydata[i].argmax()
        for k in range(len(ydata[i])):
            ydata[i][k] = (k == j)
    return ydata

In [29]:
# (i).  OneHotEncoding ydata
# (ii). Converting sparsh matrix ydata into dense form and then matrix into numpy array

def one_hot_encoded(ydata):
    ydata = OneHotEncoder().fit_transform(ydata.reshape(len(ydata),1))
    ydata = np.asarray(ydata.todense())
    return ydata

def splitData(xtrain,ytrain,type):
  if(type=='master'):
    xtrain = xtrain[(ytrain==1) | (ytrain==5)|(ytrain==7)]
    ytrain = ytrain[(ytrain==1) | (ytrain==5)|(ytrain==7)]
  elif(type=='dynamic'):
    xtrain = xtrain[(ytrain==1) | (ytrain==2)|(ytrain==3)]
    ytrain = ytrain[(ytrain==1) | (ytrain==2)|(ytrain==3)]
  elif(type=='static'):
    xtrain = xtrain[(ytrain==4) | (ytrain==5)|(ytrain==6)]
    ytrain = ytrain[(ytrain==4) | (ytrain==5)|(ytrain==6)]
  elif(type=='transition'):
    xtrain = xtrain[(ytrain==7) | (ytrain==8)|(ytrain==9)|(ytrain==10)|(ytrain==11)|(ytrain==12)]
    ytrain = ytrain[(ytrain==7) | (ytrain==8)|(ytrain==9)|(ytrain==10)|(ytrain==11)|(ytrain==12)]
  elif(type=='primary'):
    xtrain = xtrain[(ytrain==1) | (ytrain==2)|(ytrain==3)|(ytrain==4)|(ytrain==5)|(ytrain==6)]
    ytrain = ytrain[(ytrain==1) | (ytrain==2)|(ytrain==3)|(ytrain==4)|(ytrain==5)|(ytrain==6)]
  else:
    xtrain = xtrain
    ytrain = ytrain

  #xtrain,xtest,ytrain,ytest = train_test_split(xtrain,ytrain,test_size = 0.3,random_state=42)
  labels = ytrain


  ytrain = one_hot_encoded(ytrain)
  return xtrain,ytrain,labels

def splitData2(xtrain,ytrain,type):
  if(type=='master'):
    xtrain = xtrain[(ytrain==1) | (ytrain==5)|(ytrain==7)]
    ytrain = ytrain[(ytrain==1) | (ytrain==5)|(ytrain==7)]
  elif(type=='dynamic'):
    xtrain = xtrain[(ytrain==1) | (ytrain==2)|(ytrain==3)]
    ytrain = ytrain[(ytrain==1) | (ytrain==2)|(ytrain==3)]
  elif(type=='static'):
    xtrain = xtrain[(ytrain==4) | (ytrain==5)|(ytrain==6)]
    ytrain = ytrain[(ytrain==4) | (ytrain==5)|(ytrain==6)]
  elif(type=='transition'):
    xtrain = xtrain[(ytrain==7) | (ytrain==8)|(ytrain==9)|(ytrain==10)|(ytrain==11)|(ytrain==12)]
    ytrain = ytrain[(ytrain==7) | (ytrain==8)|(ytrain==9)|(ytrain==10)|(ytrain==11)|(ytrain==12)]
  elif(type=='primary'):
    xtrain = xtrain[(ytrain==1) | (ytrain==2)|(ytrain==3)|(ytrain==4)|(ytrain==5)|(ytrain==6)]
    ytrain = ytrain[(ytrain==1) | (ytrain==2)|(ytrain==3)|(ytrain==4)|(ytrain==5)|(ytrain==6)]
  else:
    xtrain = xtrain
    ytrain = ytrain

  #xtrain,xtest,ytrain,ytest = train_test_split(xtrain,ytrain,test_size = 0.3,random_state=42)

  return xtrain,ytrain


def resampleData(x,y):
  x = x.reshape(5773,768)
  df = pd.DataFrame(x)

  from imblearn.over_sampling import RandomOverSampler
  from collections import Counter
  from sklearn.datasets import make_classification
  from imblearn.over_sampling import SMOTE

  oversample = SMOTE()
  df, y = oversample.fit_resample(df, y)
  df=df.to_numpy()
  x=df.reshape(12228,128,6)
  x=df.reshape(12228,128,2,3)
  return x,y

# Data Preparation

## Loading data from files

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
# load a single file as a numpy array
def load_file(filepath):
    dataframe = read_csv(filepath, header=None, delim_whitespace=True)
    return dataframe.values

# load a list of files and return as a 3d numpy array
def load_group(filenames, prefix=''):
	loaded = list()
	for name in filenames:
		data = load_file(prefix + name)
		loaded.append(data)
	# stack group so that features are the 3rd dimension
	loaded = dstack(loaded)
	return loaded

# load a dataset group, such as train or test
def load_dataset_group(group, prefix=''):
	filepath = prefix + group + '/Inertial Signals/'
	# load all 9 files as a single array
	filenames = list()
	# total acceleration
	filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
	# body acceleration
	filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
	# body gyroscope
	filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
	# load input data
	X = load_group(filenames, filepath)
	# load class output
	y = load_file(prefix + group + '/y_'+group+'.txt')
	return X, y
#/content/drive/MyDrive/UCI HAR Dataset
# load the dataset, returns train and test X and y elements
def load_dataset(prefix=''):
    # load all train
    trainX, trainy = load_dataset_group('train', prefix + '/content/drive/MyDrive/UCI HAR Dataset/')
    # load all test
    testX, testy = load_dataset_group('test', prefix + '/content/drive/MyDrive/UCI HAR Dataset/')

    trainylabels = trainy
    testylabels = testy

    # zero-offset class values
    trainy = trainy - 1
    testy = testy - 1
    # one hot encode y
    trainy = to_categorical(trainy)
    testy = to_categorical(testy)
    print("trainX shape: ",trainX.shape)
    print("trainy shape: ", trainy.shape)
    print("testX shape: ", testX.shape)
    print("testy shape: ", testy.shape)
    print("trainylabels shape: ", trainylabels.shape)
    print("testylabels shape: ", testylabels.shape)

    return trainX, trainy, testX, testy ,trainylabels,testylabels


In [13]:
import numpy as np
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
#from keras.layers.convolutional import Conv1D
#from keras.layers.convolutional import MaxPooling1D
from tensorflow.keras.utils import to_categorical
from matplotlib import pyplot
import matplotlib.pyplot as plt
#from keras.utils.vis_utils import plot_model
import keras
import zipfile
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

trainX, trainy, testX, testy, trainylabels , testylabels = load_dataset()

trainX shape:  (7352, 128, 9)
trainy shape:  (7352, 6)
testX shape:  (2947, 128, 9)
testy shape:  (2947, 6)
trainylabels shape:  (7352, 1)
testylabels shape:  (2947, 1)


In [14]:
X = np.concatenate((trainX,testX))
Y= np.concatenate((trainy,testy))
YLabels = np.concatenate((trainylabels,testylabels))

In [15]:
X.shape,Y.shape,YLabels.shape

((10299, 128, 9), (10299, 6), (10299, 1))

In [16]:
yy = YLabels
yy.shape

(10299, 1)

In [17]:
yy =yy.reshape(10299,)

In [21]:
yy

array([5, 5, 5, ..., 2, 2, 2])

**Machine Learning Algorithms**

In [20]:
X_train = pd.read_csv('./sample_data/ucihar72features.csv', header=None)
X_train=X_train.iloc[1: , :]
Y_train = YLabels.reshape(10299,)

<ipython-input-20-35fcaabeff19>:1: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71) have mixed types. Specify dtype option on import or set low_memory=False.
  X_train = pd.read_csv('./sample_data/ucihar72features.csv', header=None)


In [22]:
folds=5

In [23]:
X.shape,X_train.shape

((10299, 128, 9), (10299, 72))

In [63]:
x1train,y1train=splitData2(X_train,Y_train,'dynamic')
x2train,y2train=splitData2(X_train,Y_train,'static')

dynamicxtrain1,dynamicytrain1,dynamiclabels1 = splitData(X,yy,'dynamic')
staticxtrain1,staticytrain1,staticlabels1 = splitData(X,yy,'static')


In [64]:
dynamicxtrain1.shape,x1train.shape

((4672, 128, 9), (4672, 72))

In [65]:
import numpy as np
from sklearn.model_selection import KFold
dynamic_fold_train=[]
dynamic_fold_test=[]
kf = KFold(n_splits=folds)
kf.get_n_splits(dynamicxtrain1)
print(kf)
for i, (train_index, test_index) in enumerate(kf.split(dynamicxtrain1)):
    dynamic_fold_train.append(train_index)
    dynamic_fold_test.append(test_index)

static_fold_train=[]
static_fold_test=[]
kf = KFold(n_splits=folds)
kf.get_n_splits(staticxtrain1)
print(kf)
for i, (train_index, test_index) in enumerate(kf.split(staticxtrain1)):
    static_fold_train.append(train_index)
    static_fold_test.append(test_index)


KFold(n_splits=5, random_state=None, shuffle=False)
KFold(n_splits=5, random_state=None, shuffle=False)


In [42]:
#X_train = np.concatenate([x1train,x2train], axis=0)
#X_test = np.concatenate([x1test,x2test], axis=0)

#Y_train = np.concatenate([y1train,y2train], axis=0)
#Y_test = np.concatenate([y1test,y2test], axis=0)

In [74]:
from sklearn.ensemble import RandomForestClassifier
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import to_categorical
from matplotlib import pyplot

accs=[]
for fold in range(folds):

  #tree_list = [200]
  #scores=[]
  #for n_trees in tree_list:
  #    rf = RandomForestClassifier(n_estimators=n_trees, random_state=40)
  #    rf.fit(X_train, Y_train)
  #    rf_pred = rf.predict(X_test)
  #    scores.append(accuracy_score(Y_test, rf_pred))
  #    print('Accuracy score TEST: {:.3f}'.format(accuracy_score(Y_test, rf_pred)*100))
  #dynamicxtrain,dynamicxtest,dynamicytrain,dynamicytest,dynamiclabels
  dynamicxtest = dynamicxtrain1[dynamic_fold_test[fold]]
  dynamicytest = dynamicytrain1[dynamic_fold_test[fold]]
  dynamicytrain = dynamicytrain1[dynamic_fold_train[fold]]
  dynamicxtrain = dynamicxtrain1[dynamic_fold_train[fold]]
  dynamiclabels = dynamiclabels1[dynamic_fold_test[fold]]

  #staticxtrain,staticxtest,staticytrain,staticytest,staticlabels

  staticxtest = staticxtrain1[static_fold_test[fold]]
  staticytest = staticytrain1[static_fold_test[fold]]
  staticytrain = staticytrain1[static_fold_train[fold]]
  staticxtrain = staticxtrain1[static_fold_train[fold]]
  staticlabels = staticlabels1[static_fold_test[fold]]


  timesteps = len(dynamicxtrain[0])
  input_dim = len(dynamicxtrain[0][0])
  n_classes =3

  print(timesteps)
  print(input_dim)
  print(len(X_train))

  epochs = 3#100
  batch_size = 16
  n_hidden = 64
  model1 = Sequential()
  # Configuring the parameters
  model1.add(LSTM(n_hidden, input_shape=(timesteps, input_dim), return_sequences=True))
  # Adding a dropout layer
  model1.add(Dropout(0.5))
  model1.add(LSTM(32))
  model1.add(Dropout(0.5))
  model1.add(Dense(n_classes, activation='softmax'))
  model1.summary()

  timesteps = len(staticxtrain[0])
  input_dim = len(staticxtrain[0][0])
  n_classes =3

  print(timesteps)
  print(input_dim)
  print(len(staticxtrain))

  epochs = 3#100
  batch_size = 16
  n_hidden = 64
  model2 = Sequential()
  # Configuring the parameters
  model2.add(LSTM(n_hidden, input_shape=(timesteps, input_dim), return_sequences=True))
  # Adding a dropout layer
  model2.add(Dropout(0.5))
  model2.add(LSTM(32))
  model2.add(Dropout(0.5))
  model2.add(Dense(n_classes, activation='softmax'))
  model2.summary()

  model1.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
  model1.fit(dynamicxtrain,
          dynamicytrain,
          batch_size=batch_size,
          #validation_data=(dynamicxtest, dynamicytest),
          epochs=epochs,
          )

  model2.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
  model2.fit(staticxtrain,
          staticytrain,
          batch_size=batch_size,
          #validation_data=(staticxtest, staticytest),
          epochs=epochs,
          )
  yexpect = np.concatenate([dynamiclabels,staticlabels], axis=0)
  merge_arr = np.concatenate([dynamicxtest, staticxtest], axis=0)
  scores=[]
  dynamiccount=0
  staticcount=0
  transitioncount=0
  dynamics = np.array([1,2,3])
  statics = np.array([4,5,6])
  transitions = np.array([7,8,9,10,11,12])
  for i in range(len(merge_arr)):#len(xtest)
      row = merge_arr[i].reshape(1,128,9)
      if(yexpect[i] in [1,2,3]):
          predict_x=model1.predict(row)

          pred=dynamics[np.argmax(predict_x,axis=1)]
          scores.append(pred[0])

      elif(yexpect[i] in [4,5,6]):
          predict_x=model2.predict(row)

          pred=statics[np.argmax(predict_x,axis=1)]
          scores.append(pred[0])


      print(i)
  print('dynamic count '+str(dynamiccount)+' static count '+str(staticcount)+' transition count '+str(transitioncount))
  from sklearn.metrics import confusion_matrix, accuracy_score


  print('Accuracy score TEST: {:.3f}'.format(accuracy_score(yexpect, scores)*100))
  accs.append(accuracy_score(yexpect, scores)*100)

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 26ms/step
1648
1/1 [==============================] - 0s 31ms/step
1649
1/1 [==============================] - 0s 26ms/step
1650
1/1 [==============================] - 0s 29ms/step
1651
1/1 [==============================] - 0s 28ms/step
1652
1/1 [==============================] - 0s 31ms/step
1653
1/1 [==============================] - 0s 29ms/step
1654
1/1 [==============================] - 0s 27ms/step
1655
1/1 [==============================] - 0s 34ms/step
1656
1/1 [==============================] - 0s 38ms/step
1657
1/1 [==============================] - 0s 26ms/step
1658
1/1 [==============================] - 0s 26ms/step
1659
1/1 [==============================] - 0s 25ms/step
1660
1/1 [==============================] - 0s 26ms/step
1661
1/1 [==============================] - 0s 27ms/step
1662
1/1 [==============================] - 0s 23ms/step
1663
1/1 [==============================] 

In [76]:
mean(accs)

90.48513302034429

**hierarchical uci har**

In [72]:


# LSTM only: 97.15
#optimal : 97.347
# rf_pred: 97.347

**Generate Dataset For traditional models**

In [ ]:
X.shape

(10299, 128, 9)

In [ ]:
x= X.reshape(10299*128,9)

In [ ]:
a=[]
for i in range(0,10299):
    a=a+[i]*128

In [ ]:
len(a)

1318272

In [ ]:
x= pd.DataFrame(x)

In [ ]:
x['id'] = a

,0,1,2,3,4,5,6,7,8,id
0,1.012817,-0.123217,0.102934,0.000181,0.010767,0.055561,0.030191,0.066014,0.022859,0
1,1.022833,-0.126876,0.105687,0.010139,0.006579,0.055125,0.043711,0.042699,0.010316,0
2,1.022028,-0.124004,0.102102,0.009276,0.008929,0.048405,0.035688,0.074850,0.013250,0
3,1.017877,-0.124928,0.106553,0.005066,0.007489,0.049775,0.040402,0.057320,0.017751,0
4,1.023680,-0.125767,0.102814,0.010810,0.006141,0.043013,0.047097,0.052343,0.002553,0
...,...,...,...,...,...,...,...,...,...,...
1318267,0.908386,-0.423054,-0.092933,-0.061667,-0.175584,0.151117,0.148295,-0.015923,0.109040,10298
1318268,0.898984,-0.392272,-0.063138,-0.070890,-0.145071,0.181814,0.143136,-0.024389,0.006547,10298
1318269,0.918862,-0.351680,-0.072539,-0.050755,-0.104717,0.173271,0.095931,-0.021024,-0.051342,10298
1318270,0.949475,-0.267526,-0.050975,-0.019807,-0.020764,0.195638,0.090708,-0.041893,-0.078877,10298


In [ ]:
from tsfresh.feature_extraction import extract_features, MinimalFCParameters, EfficientFCParameters

minimalFCParametersForTsFresh = MinimalFCParameters()
extracted_features = extract_features(x,column_id="id",default_fc_parameters = minimalFCParametersForTsFresh)

Feature Extraction: 100%|██████████| 92691/92691 [00:53<00:00, 1726.01it/s]


In [ ]:
extracted_features.columns

Index(['0__median', '0__mean', '0__standard_deviation', '0__variance',
       '0__root_mean_square', '0__maximum', '0__absolute_maximum',
       '0__minimum', '1__median', '1__mean', '1__standard_deviation',
       '1__variance', '1__root_mean_square', '1__maximum',
       '1__absolute_maximum', '1__minimum', '2__median', '2__mean',
       '2__standard_deviation', '2__variance', '2__root_mean_square',
       '2__maximum', '2__absolute_maximum', '2__minimum', '3__median',
       '3__mean', '3__standard_deviation', '3__variance',
       '3__root_mean_square', '3__maximum', '3__absolute_maximum',
       '3__minimum', '4__median', '4__mean', '4__standard_deviation',
       '4__variance', '4__root_mean_square', '4__maximum',
       '4__absolute_maximum', '4__minimum', '5__median', '5__mean',
       '5__standard_deviation', '5__variance', '5__root_mean_square',
       '5__maximum', '5__absolute_maximum', '5__minimum', '6__median',
       '6__mean', '6__standard_deviation', '6__variance',
   

In [ ]:
extracted_features = extracted_features.drop(['0__length'], axis=1)
extracted_features = extracted_features.drop(['1__length'], axis=1)
extracted_features = extracted_features.drop(['2__length'], axis=1)
extracted_features = extracted_features.drop(['3__length'], axis=1)
extracted_features = extracted_features.drop(['4__length'], axis=1)
extracted_features = extracted_features.drop(['5__length'], axis=1)
extracted_features = extracted_features.drop(['0__sum_values'], axis=1)
extracted_features = extracted_features.drop(['1__sum_values'], axis=1)
extracted_features = extracted_features.drop(['2__sum_values'], axis=1)
extracted_features = extracted_features.drop(['3__sum_values'], axis=1)
extracted_features = extracted_features.drop(['4__sum_values'], axis=1)
extracted_features = extracted_features.drop(['5__sum_values'], axis=1)

In [ ]:
extracted_features = extracted_features.drop(['6__length'], axis=1)
extracted_features = extracted_features.drop(['7__length'], axis=1)
extracted_features = extracted_features.drop(['8__length'], axis=1)
extracted_features = extracted_features.drop(['6__sum_values'], axis=1)
extracted_features = extracted_features.drop(['7__sum_values'], axis=1)
extracted_features = extracted_features.drop(['8__sum_values'], axis=1)

In [ ]:
extracted_features.to_csv('ucihar72features.csv',index=False)
